In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import tqdm

from ukbb_recessive.data_collection.dataset import RegressionDataset

In [ ]:
datasets_save_path = '.../450k/datasets/sampling/genes'

#_____________________________________________________________________________________________________________________________
s_het_paths = {
    "Roulette": ".../450k/selection_roulette/s_het_roulette_gencode-v34.csv",
}

#_____________________________________________________________________________________________________________________________
# phenotypic paths
age_children_path = ".../450k/phenotypes/age_children_data_participant.tsv" 
pca_path = ".../450k/phenotypes/PC_participant.tsv"
other_features_path = (
    ".../450k/phenotypes/phenotypes_of_interest_participant.tsv"
)

#_____________________________________________________________________________________________________________________________
# samples of interest: European & non-related
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.txt"
)

#_____________________________________________________________________________________________________________________________
# plps paths

# plps paths
variants_paths_cfg = {
    'recessive' : {
        'cohort_files': glob.glob(".../450k/RAP_output_per_chr/filtered_plps/basic/new_gene_names/new_freq/new_relatedness/chr*"), 
        'all_variants_file': ".../450k/plp_selection/basic/new_gene_names/new_freq/new_relatedness/all_chr_total_presumable_plps_HFE_final_sorted.txt"
    },
}

#_____________________________________________________________________________________________________________________________
# gene info
gencode = ".../genCode/GRCh38/v34/gencode.v34.GRCh38.txt"

#gene panel
#sampeld gene panel of ID and similar-distributed non-ID
gene_panel = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.sampled.txt")
# gene_panel.columns = ['Gene name', 'Gene panel']
gene_panel['Gene panel'].unique()

# Collect phenotypic info

In [ ]:
with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

print (f"Number of european non-related samples: {len(european_non_rel_samples)}")

features = RegressionDataset(
    age_children_path = age_children_path, 
    pca_path = pca_path, 
    other_features_path = other_features_path,
    samples_list = european_non_rel_samples, 
).collect_phenotypic_features()

# Collect s-het info

## Prepare gene panels

In [ ]:
# Create gene panels recessive
recessive_panels_dict = gene_panel.groupby('Gene panel').agg({'Gene name': lambda x: list(x)}).to_dict()['Gene name']
recessive_panels_dict['all'] = None

s_het_cfg = {
    'recessive' : recessive_panels_dict
}

for dataset_type in s_het_cfg:
    print (f"Dataset = {dataset_type}")
    for panel, gene_list in s_het_cfg[dataset_type].items():

        print (f"\tPanel={panel}, number of genes={len(gene_list) if gene_list is not None else 'NA' }")
    print()

## Collect s_het data

In [ ]:
result = {}

for s_het_key, s_het_path in s_het_paths.items():
    print (f"Collecting data for {s_het_key}\n")

    for dataset_type in s_het_cfg:
        print (f"Dataset = {dataset_type}")

        for panel, gene_list in s_het_cfg[dataset_type].items():

            print (f"\tPanel={panel}, number of genes={len(gene_list) if gene_list is not None else 'NA' }")
            print ("_____________________")
            s_het_features = RegressionDataset(
                het_occurrence_threshold=20., 
                all_plps_file=variants_paths_cfg[dataset_type]['all_variants_file'], 
                s_het_file=s_het_path,
                cohort_plps_files=variants_paths_cfg[dataset_type]['cohort_files'], 
                genes_list=gene_list,
                dataset=dataset_type
            ).collect_variant_features()
            
            # rename columns
            for col in s_het_features.columns.tolist():
                if col != 'eid':
                    s_het_features = s_het_features.rename(columns={col: f"{col}_{dataset_type}_{panel}"})

            if s_het_key not in result:
                result[s_het_key] = s_het_features
            else:
                result[s_het_key] = result[s_het_key].merge(s_het_features, on="eid", how='outer')
            print ("_____________________")

In [ ]:
# check shapes
for s_het_key in s_het_paths:
    print (f"Shape for variants features of {s_het_key}:", result[s_het_key].shape)

# Merge and fill-in NA

In [ ]:
 # fill NA for s_het with PLP but no information about s_het
fill_na_shet = True

for s_het_key in s_het_paths:
    print (f"Merging data for {s_het_key}")

    result[s_het_key] = features.merge(result[s_het_key], on='eid', how='left')

    for dataset_type in s_het_cfg:
        print (f"\tDataset = {dataset_type}")

        for panel, gene_list in s_het_cfg[dataset_type].items():
            
            columns = [f"{col}_{dataset_type}_{panel}" for col in ['s_het', 'mutations_cnt', 'has_mutation']]

            # fills NAs with 0s
            result[s_het_key][columns] = result[s_het_key][columns].fillna(0.)

            # fill NAs for s_het of recessive PLP carriers with no s_het information
            if fill_na_shet:
                result[s_het_key].loc[(result[s_het_key][f'mutations_cnt_{dataset_type}_{panel}'] > 0) & 
                                      (result[s_het_key][f's_het_{dataset_type}_{panel}'] == 0),  f's_het_{dataset_type}_{panel}'] = None

In [ ]:
# check shapes
for s_het_key in s_het_paths:
    print (f"Shape for all features of {s_het_key}:", result[s_het_key].shape)

# Shape for all features of Weghorn-drift: (376872, 231)

In [ ]:
for s_het_key in s_het_paths:
    print (f"Saving {s_het_key}", flush=True)
    result[s_het_key].to_csv(f'{datasets_save_path}/{s_het_key}.csv', sep='\t', index=False)